<a href="https://colab.research.google.com/github/thomas0913/BlockChain_BasicCase/blob/main/BlockChain_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 定義區塊鏈格式與架構

*   交易格式

In [10]:
class Transaction: #交易定義
  def __init__(self,sender,receiver,amounts,fee,message): 
    self.sender = sender    #發送者，同時check帳戶餘額是否足夠
    self.receiver = receiver  #接收者，通常直接收款
    self.amounts = amounts   #金額數
    self.fee = fee       #手續費
    self.message = message   #註記，generally for receiver

*   區塊格式



In [11]:
class Block: #區塊定義
  def __init__(self,previous_hash,difficulty,miner,miner_rewards):
    self.previous_hash = previous_hash  #前一區塊之雜湊值，為了加密
    self.hash = ''            #當前區塊雜湊值，目前區塊計算後之雜湊值
    self.difficulty = difficulty     #當前難度
    self.nonce = 0            #能解開上個區塊鎖的鑰匙
    self.timestamp = int(time.time())   #區塊產生時之時間戳，調整挖礦難度時會用到
    self.transactions = []        #交易紀錄(for all)
    self.miner = miner          #挖掘礦工(誰挖的)
    self.miner_rewards = miner_rewards  #礦工獎勵，區塊產出時分給礦工的獎勵


*   區塊鏈架構





In [12]:
class BlockChain: #區塊鏈定義
  def __init__(self):
    self.adjust_difficulty_blocks = 10 #難度調節區塊數，每多少區塊調節一次
    self.difficulty = 1        #當前難度
    self.block_time = 30        #出塊時間，理想上多久能夠產出一個區塊
    self.mining_rewards = 10      #挖礦獎勵，獎勵挖礦者的金額多寡
    self.block_limitation = 32     #區塊容量，每一區塊能夠容納的交易上限
    self.chain = []           #區塊鏈，目前鏈中儲存的所有區塊
    self.pending_transactions = []   #等待中的交易(因區塊鏈能吞吐的交易量有限)